<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Cleaning</a></span></li><li><span><a href="#Data-Split" data-toc-modified-id="Data-Split-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Split</a></span></li></ul></div>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
import pandas as pd
import numpy as np
import random
import warnings
import time
import datetime
import re
import string
import itertools
import pickle
import joblib
import nltk
import csv

from nltk.corpus import stopwords, wordnet
stop = set(stopwords.words('english'))
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from collections import Counter, defaultdict

import tensorflow as tf
import keras
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, Embedding, LSTM
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('tweets.csv', encoding='utf-8', error_bad_lines=False, sep=',')
df.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
display(df.sample(5))

,text,target
7814,I say this is as if wraith WOULDNT absolutely ...,0
3095,Titan: My whole life is a lie! I just found ou...,0
3952,The streets of Iran are filled with thankful p...,0
6208,Goals this season👑 Rashford 19 Martial 11 Gree...,0
5900,🛑 Democrats are flooding our country with ILLE...,0


In [6]:
df.shape

(11370, 2)

In [7]:
df['target'].value_counts()

0    9256
1    2114
Name: target, dtype: int64

## Data Cleaning

In [8]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

df['clean_text'] = df['text'].apply(lambda x: remove_url(str(x)))
df['clean_text'] = df['clean_text'].apply(lambda x: remove_emoji(str(x)))
df['clean_text'] = df['clean_text'].apply(lambda x: remove_html(str(x)))
df['clean_text'] = df['clean_text'].apply(lambda x: remove_punct(str(x)))
df['clean_text'] = df['clean_text'].apply(word_tokenize)
df['clean_text'] = df['clean_text'].apply(lambda x: [word.lower() for word in x])
df['clean_text'] = df['clean_text'].apply(lambda x: [word for word in x if word not in stop])
df['clean_text'] = df['clean_text'].apply(nltk.tag.pos_tag)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
df['clean_text'] = df['clean_text'].apply(
    lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
wnl = WordNetLemmatizer()
df['clean_text'] = df['clean_text'].apply(
    lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
df['clean_text'] = df['clean_text'].apply(
    lambda x: [word for word in x if word not in stop])
df['clean_text'] = [' '.join(map(str, l)) for l in df['clean_text']]

In [9]:
display(df.sample(2))

,text,target,clean_text
5202,If you convert one kilogram of mass into energ...,0,convert one kilogram mass energy create 000009...
9357,Bushfire smoke plume destined to reach Austral...,1,bushfire smoke plume destine reach australia c...


In [ ]:
# df.to_csv('tweets_processed.csv')

## Data Preparation

In [10]:
disaster_initial = list(df[df['target'] == 1]['clean_text'])
non_disaster_initial = list(df[df['target'] == 0]['clean_text'])

# Selecting only 20 samples for siamese model
disaster = disaster_initial[:20]
non_disaster = non_disaster_initial[:20]

# Creating pairs of data for siamese training => label 1 if pairs from same class otherwise 0
df2 = pd.DataFrame(columns=['text1', 'text2', 'label'])

for data in disaster:
  data1 = data
  data2 = random.choice(disaster)
  data3 = random.choice(non_disaster)

  df2.loc[len(df2)] = [data1, data2, 1]
  df2.loc[len(df2)] = [data1, data3, 0]


for data in non_disaster:
  data1 = data
  data2 = random.choice(non_disaster)
  data3 = random.choice(disaster)
  
  df2.loc[len(df2)] = [data1, data2, 1]
  df2.loc[len(df2)] = [data1, data3, 0]


In [11]:
df2.shape

(80, 3)

In [12]:
display(df2.sample(5))

,text1,text2,label
41,lord jesus love brings freedom pardon fill hol...,turn blind eye icident set ablaze 50 house h…,0
31,another arson njikomboyonwr ambazombies yester...,amen set whole system ablaze man,0
2,telangana section 144 impose bhainsa january 1...,arsonist set car ablaze dealership,1
62,kind true sadly,back neck still fuck accident,1
7,arsonist set car ablaze dealership,national security minister kan dapaahs side ch...,0


## Data Split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(df2[['text1', 'text2']], df2['label'], test_size=0.2, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(64, 2) (16, 2) (64,) (16,)


In [14]:
X_train['text'] = X_train[['text1', 'text2']].apply(lambda x: str(x[0])+" "+str(x[1]), axis=1)

## Glove Embeddings

In [15]:
t = Tokenizer()
t.fit_on_texts(X_train['text'].values)

X_train['text1'] = X_train['text1'].astype(str)
X_train['text2'] = X_train['text2'].astype(str)
X_val['text1'] = X_val['text1'].astype(str)
X_val['text2'] = X_val['text2'].astype(str)

train_q1_seq = t.texts_to_sequences(X_train['text1'].values)
train_q2_seq = t.texts_to_sequences(X_train['text2'].values)
val_q1_seq = t.texts_to_sequences(X_val['text1'].values)
val_q2_seq = t.texts_to_sequences(X_val['text2'].values)

max_len = 200
train_q1_seq = pad_sequences(train_q1_seq, maxlen=max_len, padding='post')
train_q2_seq = pad_sequences(train_q2_seq, maxlen=max_len, padding='post')
val_q1_seq = pad_sequences(val_q1_seq, maxlen=max_len, padding='post')
val_q2_seq = pad_sequences(val_q2_seq, maxlen=max_len, padding='post')

In [16]:
#https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
embeddings_index = {}
f = open('drive/My Drive/Glove/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [17]:
not_present_list = []
vocab_size = len(t.word_index) + 1
print('Loaded %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((vocab_size, len(embeddings_index['no'])))
for word, i in t.word_index.items():
    if word in embeddings_index.keys():
        embedding_vector = embeddings_index.get(word)
    else:
        not_present_list.append(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.zeros(300)

Loaded 400000 word vectors.


In [18]:
print(embedding_matrix.shape)

(317, 300)


## Siamese Model

In [19]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

In [59]:
def build_network():

  network = Sequential()
  network.add(Embedding(name="synopsis_embedd",input_dim =len(t.word_index)+1, 
                       output_dim=len(embeddings_index['no']),weights=[embedding_matrix], 
                       input_length=train_q1_seq.shape[1],trainable=False))
  network.add(LSTM(64,return_sequences=True, activation="relu"))
  network.add(Flatten())
  network.add(Dense(4096, activation='relu',
                  kernel_regularizer=l2(1e-3),
                  kernel_initializer='he_uniform'))
  
  network.add(Dense(317, activation=None,
                  kernel_regularizer=l2(1e-3),
                  kernel_initializer='he_uniform'))
  
  #Force the encoding to live on the d-dimentional hypershpere
  network.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))

  return network

In [60]:
input_1 = Input(shape=(train_q1_seq.shape[1],))
input_2 = Input(shape=(train_q2_seq.shape[1],))

network = build_network()

encoded_input_1 = network(input_1)
encoded_input_2 = network(input_2)

distance = Lambda(euclidean_distance)([encoded_input_1, encoded_input_2])

# Connect the inputs with the outputs
model = Model([input_1, input_2], distance)

model.compile(loss=contrastive_loss, optimizer=Adam(0.001))

In [47]:
y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')

In [48]:
model.fit([train_q1_seq,train_q2_seq],y_train.reshape(-1,1), epochs = 5, 
          batch_size=64,validation_data=([val_q1_seq, val_q2_seq],y_val.reshape(-1,1)))

Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 9.1177 - val_loss: 8.2265
Epoch 2/5
1/1 [==============================] - 2s 2s/step - loss: 8.3272 - val_loss: 7.3562
Epoch 3/5
1/1 [==============================] - 2s 2s/step - loss: 7.4295 - val_loss: 6.5293
Epoch 4/5
1/1 [==============================] - 2s 2s/step - loss: 6.5605 - val_loss: 5.7697
Epoch 5/5
1/1 [==============================] - 2s 2s/step - loss: 5.7392 - val_loss: 5.1012


In [49]:
# Save model for further use
# serialize model to JSON
model_json = model.to_json()
with open("siamesemodel-contrastive-loss.json", "w") as json_file:
    json_file.write(model_json)
#serialize weights to HDF5
model.save_weights("siamesemodel-contrastive-loss.h5")
print("Saved model to disk")

# load json and create model
# json_file = open('siamesemodel-contrastive-loss.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("siamesemodel-contrastive-loss.h5")
# print("Loaded model from disk")

Saved model to disk


## Prediction

### Non Disaster Tweet

In [50]:
non_disaster_initial[122]

'usdjpy despite richter scale overbought fxstreet line swissquote bullish long jason sen mr ambulance chaser b…'

In [51]:
prediction_data = "usdjpy despite richter scale overbought fxstreet line swissquote bullish long jason sen mr ambulance chaser b…"
prediction_vector = t.texts_to_sequences([prediction_data])
prediction_vector = pad_sequences(prediction_vector,maxlen=200)

assistant_data = disaster[11]
assistant_vector = t.texts_to_sequences([assistant_data])
assistant_vector = pad_sequences(assistant_vector,maxlen=200)

model.predict([prediction_vector, assistant_vector])

array([[0.49869883]], dtype=float32)

In [52]:
prediction_data = "usdjpy despite richter scale overbought fxstreet line swissquote bullish long jason sen mr ambulance chaser b…"
prediction_vector = t.texts_to_sequences([prediction_data])
prediction_vector = pad_sequences(prediction_vector,maxlen=200)

assistant_data = non_disaster[11]
assistant_vector = t.texts_to_sequences([assistant_data])
assistant_vector = pad_sequences(assistant_vector,maxlen=200)

model.predict([prediction_vector, assistant_vector])

array([[0.36637673]], dtype=float32)

### Disaster Tweet

In [53]:
disaster_initial[12]

'cameroon bir soldier 05012020 invaded southerncameroons village kimar set ablaze total of…'

In [54]:
prediction_data = "cameroon bir soldier 05012020 invaded southerncameroons village kimar set ablaze total of"
prediction_vector = t.texts_to_sequences([prediction_data])
prediction_vector = pad_sequences(prediction_vector,maxlen=200)

assistant_data = disaster[11]
assistant_vector = t.texts_to_sequences([assistant_data])
assistant_vector = pad_sequences(assistant_vector,maxlen=200)

model.predict([prediction_vector, assistant_vector])

array([[0.38583857]], dtype=float32)

In [55]:
prediction_data = "cameroon bir soldier 05012020 invaded southerncameroons village kimar set ablaze total of"
prediction_vector = t.texts_to_sequences([prediction_data])
prediction_vector = pad_sequences(prediction_vector,maxlen=200)

assistant_data = non_disaster[11]
assistant_vector = t.texts_to_sequences([assistant_data])
assistant_vector = pad_sequences(assistant_vector,maxlen=200)

model.predict([prediction_vector, assistant_vector])

array([[0.46985233]], dtype=float32)